In [74]:
import pickle
from sklearn import model_selection, metrics
import numpy as np
import pandas as pd

AUDIO_EMBEDDINGS = '/Users/agnieszkalenart/Documents/mannheim/master_thesis/cmn_audio_opensmile_new.pickle'
trainID = pickle.load(open("/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/CMN_wav2vec2/IEMOCAP/data/trainID_new.pkl",'rb'), encoding="latin1")
testID = pickle.load(open("/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/CMN_wav2vec2/IEMOCAP/data/testID_new.pkl",'rb'), encoding="latin1")
valID,_ = model_selection.train_test_split(testID, test_size=.4, random_state=1227)

Preprocess data

In [75]:
audio_emb = pickle.load(open(AUDIO_EMBEDDINGS, 'rb'), encoding="latin1")
df = pd.DataFrame.from_dict(audio_emb, orient='index')
transcripts, labels, own_historyID, other_historyID, own_historyID_rank, other_historyID_rank = pickle.load(open("/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/CMN_wav2vec2/IEMOCAP/data/dataset.pkl",'rb'), encoding="latin1")

In [16]:
df.shape

(1373, 6373)

In [13]:
len(labels)

7433

In [76]:
# filter 'xxx', 'sur', 'oth', 'dis', 'fea' from labels out
todrop_values = ['xxx', 'sur', 'oth', 'dis', 'fea']
todrop_keys = [key for key, value in labels.items() if value in todrop_values]
for key in todrop_keys:
    if key in labels:
        del labels[key]

In [77]:
# create embedding for filtered labels
dict_filtered_out = {}
for key in todrop_keys:
    if key in df.index:
        dict_filtered_out[key] = np.zeros(100)

In [78]:
# filter 'xxx', 'sur', 'oth', 'dis', 'fea' from df out
todrop_values = ['xxx', 'sur', 'oth', 'dis', 'fea']
# Find the common indexes
common_indexes = set(df.index).intersection(todrop_keys)
# Drop the specified indexes
df = df.drop(common_indexes)
for key in todrop_keys:
    if key in audio_emb:
        del audio_emb[key]

In [79]:
label_idx = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5}
labels_array = np.asarray([label_idx[labels[ID]] for ID in audio_emb.keys()])

In [4]:
labels

{'Ses01F_script02_2_F042': 'hap',
 'Ses01F_script02_2_F043': 'hap',
 'Ses01F_script02_2_F040': 'sad',
 'Ses01F_script02_2_F041': 'neu',
 'Ses01F_script02_2_F046': 'sad',
 'Ses01F_script02_2_F047': 'neu',
 'Ses01F_script02_2_F044': 'hap',
 'Ses01F_script02_2_F045': 'hap',
 'Ses05F_script01_1_M010': 'fru',
 'Ses05F_script01_1_M011': 'fru',
 'Ses05F_script01_1_M012': 'fru',
 'Ses05F_script01_1_M013': 'ang',
 'Ses05F_script01_1_M014': 'fru',
 'Ses05F_script01_1_M015': 'fru',
 'Ses05F_script01_1_M016': 'sad',
 'Ses05F_script01_1_M017': 'fru',
 'Ses05M_script02_2_F030': 'sad',
 'Ses05M_script02_2_F031': 'sad',
 'Ses05M_script02_2_F032': 'sad',
 'Ses05M_script02_2_F033': 'xxx',
 'Ses05M_script02_2_F034': 'hap',
 'Ses05M_script02_2_F035': 'xxx',
 'Ses05M_script02_2_F036': 'xxx',
 'Ses05M_script02_2_F037': 'hap',
 'Ses05M_script02_2_F038': 'hap',
 'Ses05F_script03_2_F041': 'ang',
 'Ses05F_script03_2_F040': 'ang',
 'Ses01M_script03_1_M028': 'exc',
 'Ses01M_script03_1_M029': 'exc',
 'Ses01M_scrip

Neural Net without cross-validation

In [80]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# Split the data into features and labels
features = df
labels = pd.Series(labels_array)

# Split the data into training, validation and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


# Standardize the feature values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Define the neural network model
model = Sequential()
model.add(Dense(100, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(labels.unique()), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Predict the emotions on the testing set
predict_x = model.predict(X_test)
classes_x=np.argmax(predict_x,axis=1)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, classes_x)
print(f'Accuracy: {accuracy}')

Epoch 1/10
28/28 [==============================] - 0s 6ms/step - loss: 1.9436 - accuracy: 0.3872 - val_loss: 1.5181 - val_accuracy: 0.4727
Epoch 2/10
28/28 [==============================] - 0s 3ms/step - loss: 0.7603 - accuracy: 0.7574 - val_loss: 1.6424 - val_accuracy: 0.5055
Epoch 3/10
28/28 [==============================] - 0s 3ms/step - loss: 0.3176 - accuracy: 0.9134 - val_loss: 2.1460 - val_accuracy: 0.4218
Epoch 4/10
28/28 [==============================] - 0s 4ms/step - loss: 0.1489 - accuracy: 0.9567 - val_loss: 1.9608 - val_accuracy: 0.4727
Epoch 5/10
28/28 [==============================] - 0s 3ms/step - loss: 0.0641 - accuracy: 0.9863 - val_loss: 1.8147 - val_accuracy: 0.4982
Epoch 6/10
28/28 [==============================] - 0s 4ms/step - loss: 0.0249 - accuracy: 0.9989 - val_loss: 1.8516 - val_accuracy: 0.4873
Epoch 7/10
28/28 [==============================] - 0s 3ms/step - loss: 0.0124 - accuracy: 1.0000 - val_loss: 1.9128 - val_accuracy: 0.4982
Epoch 8/10
28/28 [==

Neural Net with Cross-validation

In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K


# Split the data into features and labels
features = df
labels = pd.Series(labels_array).reset_index(drop=True)
indices = df.index
# Split the data into training and testing sets
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, indices, test_size=0.2, random_state=42)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Standardize the feature values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the neural network model
def create_model():
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(100, activation='relu'))    
    model.add(Dense(len(labels.unique()), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Perform cross-validation
num_folds = 10
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
cv_scores = []
best_model = None
best_score = 0

for train_index, val_index in skf.split(X_train, y_train):
    train_features, val_features = X_train[train_index], X_train[val_index]
    train_labels, val_labels = y_train[train_index], y_train[val_index]
    
    model = create_model()
    model.fit(train_features, train_labels, epochs=20, batch_size=32, verbose=0)
    
    val_pred = model.predict(val_features)
    classes_x=np.argmax(val_pred,axis=1)
    fold_score = accuracy_score(val_labels, classes_x)

    if fold_score > best_score:
        best_score = fold_score
        best_model = model

    cv_scores.append(fold_score)

# Print the cross-validation scores
print('Cross-Validation Scores:', cv_scores)
print('Average Accuracy:', np.mean(cv_scores))


# Create a new model that outputs the activations of the last layer
representation_model = Model(inputs=best_model.input, outputs=best_model.layers[-2].output)

# Extract the representations for each example
train_representations = representation_model.predict(X_train)
test_representations = representation_model.predict(X_test)

# Create dict representations
representations = {}
representations.update({idx: rep for idx, rep in zip(indices_train, train_representations)})
representations.update({idx: rep for idx, rep in zip(indices_test, test_representations)})
representations.update(dict_filtered_out)


# Save the representations as a pickle file
with open('representations.pkl', 'wb') as f:
    pickle.dump(representations, f)

# Predict the emotions on the testing set
y_pred = best_model.predict(X_test)
classes_x = np.argmax(y_pred, axis=1)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, classes_x)
print(f'Test Accuracy: {accuracy}')

4/4 [==============================] - 0s 941us/step
Cross-Validation Scores: [0.509090909090909, 0.5909090909090909, 0.5545454545454546, 0.5545454545454546, 0.509090909090909, 0.5909090909090909, 0.5909090909090909, 0.4818181818181818, 0.5321100917431193, 0.5321100917431193]
Average Accuracy: 0.544603836530442
9/9 [==============================] - 0s 1ms/step
Test Accuracy: 0.509090909090909


Auto-encoder

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Split the data into features and labels
features = df
labels = pd.Series(labels_array).reset_index(drop=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Standardize the feature values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the dimensions of the autoencoder
input_dim = X_train.shape[1]
encoding_dim = 100

# Define the autoencoder model
input_layer = Input(shape=(input_dim,))
encoder_layer = Dense(encoding_dim, activation='relu')(input_layer)
decoder_layer = Dense(input_dim, activation='relu')(encoder_layer)
autoencoder = Model(inputs=input_layer, outputs=decoder_layer)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder model
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_test, X_test))

# Create a new model that represents the encoder part of the autoencoder
encoder = Model(inputs=input_layer, outputs=encoder_layer)

# Obtain the dense representation of the features
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)

# Use the encoded features for emotion classification
model = Sequential()
model.add(Dense(100, activation='relu', input_shape=(encoding_dim,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(labels.unique()), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_encoded, y_train, epochs=10, batch_size=32, validation_data=(X_test_encoded, y_test))

# Predict the emotions on the testing set
y_pred = model.predict(X_test_encoded)
classes_x=np.argmax(y_pred,axis=1)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, classes_x)
print(f'Accuracy: {accuracy}')


Epoch 1/10
35/35 [==============================] - 0s 5ms/step - loss: 0.9084 - val_loss: 1.2601
Epoch 2/10
35/35 [==============================] - 0s 4ms/step - loss: 0.8251 - val_loss: 1.2281
Epoch 3/10
35/35 [==============================] - 0s 4ms/step - loss: 0.7925 - val_loss: 1.2126
Epoch 4/10
35/35 [==============================] - 0s 4ms/step - loss: 0.7718 - val_loss: 1.1976
Epoch 5/10
35/35 [==============================] - 0s 4ms/step - loss: 0.7522 - val_loss: 1.1857
Epoch 6/10
35/35 [==============================] - 0s 4ms/step - loss: 0.7363 - val_loss: 1.1781
Epoch 7/10
35/35 [==============================] - 0s 5ms/step - loss: 0.7234 - val_loss: 1.1730
Epoch 8/10
35/35 [==============================] - 0s 4ms/step - loss: 0.7117 - val_loss: 1.1696
Epoch 9/10
35/35 [==============================] - 0s 4ms/step - loss: 0.7024 - val_loss: 1.1655
Epoch 10/10
9/9 [==============================] - 0s 662us/step
Epoch 1/10
35/35 [==============================] - 0